In [2]:
import numpy as np
import pandas as pd
import sqlite3 as sql
from sklearn.preprocessing import MinMaxScaler
from ipywidgets import interact # para análisis interactivo
from sklearn import neighbors # basado en contenido un solo producto consumido
import joblib

from surprise import Reader, Dataset
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline
from surprise.model_selection import train_test_split

# conectar base de datos
conn=sql.connect('Data/db_moviesi.db')
cur=conn.cursor()

# ver tablas disponibles en base de datos ###
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
cur.fetchall()


[('ratings',),
 ('movies',),
 ('usuarios_sel',),
 ('movies_sel',),
 ('ratings_final',),
 ('full_ratings',),
 ('full_ratings_sin_dup',),
 ('reco',)]

In [11]:
df = pd.read_sql("SELECT * FROM full_ratings", conn)
df

,user_id,movie_id,rating,movie_title,movie_genres,movie_clean_title,movie_year,fecha_nueva
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story (1995),1995.0,2000-07-30
1,1,3,4.0,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men (1995),1995.0,2000-07-30
2,1,6,4.0,Heat (1995),Action|Crime|Thriller,Heat (1995),1995.0,2000-07-30
3,1,47,5.0,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,Seven (a.k.a. Se7en) (1995),1995.0,2000-07-30
4,1,50,5.0,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,"Usual Suspects, The (1995)",1995.0,2000-07-30
...,...,...,...,...,...,...,...,...
70524,609,892,3.0,Twelfth Night (1996),Comedy|Drama|Romance,Twelfth Night (1996),1996.0,1996-11-05
70525,609,1056,3.0,Jude (1996),Drama,Jude (1996),1996.0,1996-11-05
70526,609,1059,3.0,William Shakespeare's Romeo + Juliet (1996),Drama|Romance,William Shakespeare's Romeo + Juliet (1996),1996.0,1996-11-05
70527,609,1150,4.0,"Return of Martin Guerre, The (Retour de Martin...",Drama,"Return of Martin Guerre, The (Retour de Martin...",1982.0,1996-11-05


# <b>4. Sistemas de recomendación basado en el contenido de cada usuario</b>
Este sistema se enfoca en analizar las interacciones pasadas de cada usuario con las películas para generar recomendaciones ajustadas a sus preferencias individuales. Utiliza el historial de visualización y las calificaciones otorgadas por el usuario para identificar patrones y gustos específicos. A partir de este análisis, el sistema sugiere películas que son más propensas a interesar al usuario, asegurando una experiencia de visualización más satisfactoria. 
La frecuencia de actualización de estas recomendaciones es mensual, lo que permite que el sistema tenga tiempo suficiente para detectar cambios en las preferencias del usuario. Este enfoque asegura que las recomendaciones sean relevantes y continúen mejorando a lo largo del tiempo, adaptándose a las nuevas elecciones y tendencias de visualización del usuario.

In [6]:
# Eliminar las columnas del DataFrame
movies = df.drop(columns=['user_id','rating', 'movie_title', 'fecha_nueva'])
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70529 entries, 0 to 70528
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   movie_id           70529 non-null  int64  
 1   movie_genres       70529 non-null  object 
 2   movie_clean_title  70529 non-null  object 
 3   movie_year         70518 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 2.2+ MB


In [7]:
movies['movie_year'] = movies['movie_year'].astype(int)


IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer